# Import Modules

In [1]:
import pandas as pd
import numpy as np

import time
import sys

from ML_toolbox import data_describe as dd

from recommand_sys_svd import svd_recommand_sys as srs

# Load Data

In [2]:
path = ['data/', 'result/']
file = ['client', 'all_data']
res_file = ['group_top_n_brand', 'group_top_n_category', 'group_top_n_pair']
top_file = ['top_n_brand', 'top_n_category', 'top_n_cb']

df_client = pd.read_csv(f'{path[0]}{file[0]}.csv', encoding='utf8')
df_all = pd.read_csv(f'{path[0]}{file[1]}.csv', encoding='utf8')

print(df_client.shape)
print(df_client.columns)

print(df_all.shape)
print(df_all.columns)

cat_map = {1:'去角質', 2:'沐浴', 3:'其他', 4:'卸妝', 5:'底妝',
           6:'保濕', 7:'持妝', 8:'洗髮', 9:'洗顏', 10:'眉毛',
           11:'美白', 12:'眼線', 13:'睫毛', 14:'補妝', 15:'遮瑕', 16:'護唇'}

(445, 9)
Index(['member_id', 'name', 'age', 'regist_days', 'last_purchaseday',
       'tol_purchase_price', 'tol_purchase_times', 'gender',
       'membership_source'],
      dtype='object')
(649, 21)
Index(['cart_id', 'member_id', 'retailer', 'address', 'channel', 'category',
       'brand', 'cnt', 'per_price', 'shipping_fee', 'discount', 'payment',
       'attribute', 'additional_purchase', 'age', 'regist_days',
       'last_purchaseday', 'tol_purchase_price', 'tol_purchase_time', 'gender',
       'membership_source'],
      dtype='object')


In [3]:
df_group = df_all[['member_id','age', 'gender', 'category', 'brand', 'cnt', 'per_price']]
df_group['score'] = df_group['cnt']*df_group['per_price']
df_group.head()

C:\Users\ASUS\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,member_id,age,gender,category,brand,cnt,per_price,score
0,5105005930,0,-1,10,A,1,558,558
1,5503419373,0,-1,8,K,2,432,864
2,5503419373,0,-1,2,K,1,288,288
3,5506618770,0,-1,13,A,1,620,620
4,5501147671,0,-1,2,K,2,288,576


In [4]:
df_group.isnull().any()

member_id    False
age          False
gender       False
category     False
brand        False
cnt          False
per_price    False
score        False
dtype: bool

---
# Table of user group

In [5]:
# make column age be level

bins = [0, 19, 22, 23, 32, 40, 100]
group_name = [0, 1, 2, 3, 4, 5]
df_group.age = pd.cut(df_group.age, bins, labels = group_name, right=False)
df_group['age'] = df_group['age'].astype(int)
print(df_group.age.unique())
df_group.isnull().any()

[0 2 4 3 5 1]


C:\Users\ASUS\AppData\Local\Programs\Python\Python36\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\ASUS\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


member_id    False
age          False
gender       False
category     False
brand        False
cnt          False
per_price    False
score        False
dtype: bool

In [6]:
df_group['age'] = df_group['age'].astype(str)
df_group['gender'] = df_group['gender'].astype(str)

df_group['group'] = df_group['age'] + '_' + df_group['gender']

df_group.head()

C:\Users\ASUS\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\ASUS\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\ASUS\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

,member_id,age,gender,category,brand,cnt,per_price,score,group
0,5105005930,0,-1,10,A,1,558,558,0_-1
1,5503419373,0,-1,8,K,2,432,864,0_-1
2,5503419373,0,-1,2,K,1,288,288,0_-1
3,5506618770,0,-1,13,A,1,620,620,0_-1
4,5501147671,0,-1,2,K,2,288,576,0_-1


## 1. group & brand

In [7]:
df_group_brand = df_group[['group', 'brand', 'score']]
df_group_brand = df_group_brand.groupby(['group', 'brand']).score.sum()
df_group_brand = df_group_brand.reset_index(inplace=False)
print(df_group_brand.shape)
df_group_brand.head()

(42, 3)


,group,brand,score
0,0_-1,A,126417
1,0_-1,B,4764
2,0_-1,C,8038
3,0_-1,D,1480
4,0_-1,F,2715


In [8]:
# top3 to recommand for group

df_group_brand_res = srs.top_recommand(df_group_brand, 'group', 'score', 3, groupby_title='group')
df_group_brand_res.head()

,group,brand,score
40,5_0,K,6399
37,5_0,A,4630
38,5_0,B,1159
31,4_0,A,6950
34,4_0,K,1998


In [9]:
# to csv 
df_group_brand_res.to_csv(f'{path[1]}{res_file[0]}.csv', encoding='utf8', index=False)

## 2. group & category

In [10]:
df_group_cat = df_group[['group', 'category', 'score']]
df_group_cat = df_group_cat.groupby(['group', 'category']).score.sum()
df_group_cat = df_group_cat.reset_index(inplace=False)
print(df_group_cat.shape)
df_group_cat.head()

(66, 3)


,group,category,score
0,0_-1,1,6989
1,0_-1,2,5066
2,0_-1,3,390
3,0_-1,4,2202
4,0_-1,5,2310


In [11]:
# top3 to recommand for group

df_group_cat_res = srs.top_recommand(df_group_cat, 'group', 'score', 3, groupby_title='group')

df_group_cat_res['category'] = df_group_cat_res['category'].map(cat_map)
df_group_cat_res.head()

,group,category,score
60,5_0,洗髮,3570
59,5_0,保濕,3039
63,5_0,美白,1884
52,4_0,洗髮,3570
50,4_0,保濕,2879


In [12]:
# to csv 
df_group_cat_res.to_csv(f'{path[1]}{res_file[1]}.csv', encoding='utf8', index=False)

## 3. group & category+brand 

In [13]:
# count each member category pair purchase times
df_cb_rating = df_group[['group','category','brand','score']].groupby(['group','category','brand'])['score'].sum()

df_cb_rating = df_cb_rating.reset_index(inplace=False, level=['group','category','brand'])  
# print(df_cb_rating.shape)
df_cb_rating.category = df_cb_rating.category.astype(str)

df_cb_rating['item'] = df_cb_rating.category + df_cb_rating.brand

df_cb_rating = df_cb_rating.reindex(columns=['group', 'item','category', 'brand', 'score'])


# This table is used to record the new id's brand and category 
cat_brand_table = df_cb_rating.drop(['group', 'score'],axis=1)
cat_brand_table.drop_duplicates()

# This table is used to do recommand
df_cb_rating_new = df_cb_rating.drop(['category','brand'], axis=1)

df_cb_rating_new.head()

,group,item,score
0,0_-1,1C,6989
1,0_-1,2K,5066
2,0_-1,3B,0
3,0_-1,3L,390
4,0_-1,3others,0


In [14]:
# top3 to recommand for group
df_cb_res = srs.top_recommand(df_cb_rating_new, 'group', 'score', 3, groupby_title='group')
df_cb_res.head()

,group,item,score
91,5_0,8A,3570
90,5_0,6K,1979
94,5_0,11K,1884
81,4_0,8A,3570
82,4_0,13A,1700


In [15]:
# reorganize data

df_cb_res = df_cb_res.merge(cat_brand_table, how='left', on='item').drop_duplicates()
df_cb_res.drop(columns=['item'], inplace=True)

df_cb_res['category'] = df_cb_res['category'].astype(int)
df_cb_res['category'] = df_cb_res['category'].map(cat_map)

df_cb_res.head()

,group,score,category,brand
0,5_0,3570,洗髮,A
6,5_0,1979,保濕,K
12,5_0,1884,美白,K
15,4_0,3570,洗髮,A
21,4_0,1700,睫毛,A


In [16]:
# to csv 
df_cb_res.to_csv(f'{path[1]}{res_file[2]}.csv', encoding='utf8', index=False)

---
# Example: Insert test

In [17]:
# Load data 
path = ['data/', 'result/']
file = ['client', 'all_data']
res_file = ['group_top_n_brand', 'group_top_n_category', 'group_top_n_pair']

df_group_brand_res = pd.read_csv(f'{path[1]}{res_file[0]}.csv', encoding='utf8')
df_group_cat_res = pd.read_csv(f'{path[1]}{res_file[1]}.csv', encoding='utf8')
df_cb_res = pd.read_csv(f'{path[1]}{res_file[2]}.csv', encoding='utf8')

In [18]:
# transformate data

def insert_trans(insert_data, bins):
    result=['']
    for i in range(len(bins)):
        if insert_data[0] in range(bins[i], bins[i+1]):
            result[0] = i
            break

    if insert_data[1] == 'female': result.append(0)
    elif insert_data[1] == 'male': result.append(1)
    else: result.append(-1)

    group_id = str(result[0])+'_'+str(result[1])
    
    return group_id

In [19]:
# get data

def tar_recommand(df_res, user_title, tar_user, tar_title_list):
    '''
    ==============================================
    
    ==============================================
    >* df_res: the dataframe with users and top n recommand target
    >* user_title: the name of column where user in 
    >* tar_user: the target user 
    >* tar_title_list: list of target items
    '''
    return df_res[df_res[user_title]==tar_user][tar_title_list]


In [20]:
# transformate data

A = (15, 'female') 
bins = [0, 19, 22, 23, 32, 40, 100]

group_id = insert_trans(A, bins)
group_id

'0_0'

In [21]:
# get data: brand
tar_brand = tar_recommand(df_group_brand_res, 'group', group_id, ['brand'])
tar_brand

,brand
15,K
16,A
17,C


In [22]:
tar_brand.brand[tar_brand.index[0]]

'K'

In [23]:
# get data: category
tar_recommand(df_group_cat_res, 'group', group_id, ['category'])

,category
15,保濕
16,洗髮
17,去角質


In [24]:
# get data: category & brand pair
# group_cb['category'] = group_cb['category'].astype(int)
group_cb = tar_recommand(df_cb_res, 'group', group_id, ['category', 'brand'])
group_cb['category'] = group_cb['category'].map(cat_map)
group_cb

,category,brand
15,NaN,K
16,NaN,A
17,NaN,A


---
# Top n to recommand

In [25]:
top_tab = df_all[['category', 'brand', 'cnt', 'per_price']]
top_tab['score'] = top_tab['cnt']*top_tab['per_price']
top_tab[['category', 'brand', 'score']].groupby(['category', 'brand'])['score'].sum()
top_tab.reset_index(inplace=True, drop=True)
# top_tab.head()

C:\Users\ASUS\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## 1. Top n brand 

In [26]:
top_n_brand = top_tab[['brand', 'score']].groupby('brand')['score'].sum().sort_values(ascending=False)
top_n_brand.head(3)

brand
A    175482
K    116627
M     13856
Name: score, dtype: int64

In [27]:
top_n_brand.to_csv(f'{path[1]}{top_file[0]}.csv', encoding='utf8')

## 2. top n category

In [28]:
top_n_category = top_tab[['category', 'score']].groupby('category')['score'].sum().sort_values(ascending=False)

top_n_category.index= top_n_category.index.map(cat_map)

top_n_category.head(3)

category
保濕    133322
洗髮    105526
睫毛     48242
Name: score, dtype: int64

In [29]:
pd.DataFrame(top_n_category).to_csv(f'{path[1]}{top_file[1]}.csv', encoding='utf8')

## 3. top n category & brand pair

In [30]:
df = top_tab[['category', 'brand', 'score']].drop_duplicates()  
top_n_cb = df.sort_values(['score'],ascending=False)

top_n_cb['category'] = top_n_cb['category'].map(cat_map)

top_n_cb.head(3)

,category,brand,score
80,洗髮,A,3499
112,睫毛,M,2430
89,洗髮,A,2250


In [31]:
top_n_cb.to_csv(f'{path[1]}{top_file[2]}.csv', encoding='utf8', index=False)